In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.\
        builder.\
        appName("pyspark-notebook").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "2g").\
        getOrCreate()

22/02/17 19:54:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
df = spark.read.option("header", "true").csv("data/final_dataset.csv")
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- Flow ID: string (nullable = true)
 |-- Src IP: string (nullable = true)
 |-- Src Port: string (nullable = true)
 |-- Dst IP: string (nullable = true)
 |-- Dst Port: string (nullable = true)
 |-- Protocol: string (nullable = true)
 |-- Timestamp: string (nullable = true)
 |-- Flow Duration: string (nullable = true)
 |-- Tot Fwd Pkts: string (nullable = true)
 |-- Tot Bwd Pkts: string (nullable = true)
 |-- TotLen Fwd Pkts: string (nullable = true)
 |-- TotLen Bwd Pkts: string (nullable = true)
 |-- Fwd Pkt Len Max: string (nullable = true)
 |-- Fwd Pkt Len Min: string (nullable = true)
 |-- Fwd Pkt Len Mean: string (nullable = true)
 |-- Fwd Pkt Len Std: string (nullable = true)
 |-- Bwd Pkt Len Max: string (nullable = true)
 |-- Bwd Pkt Len Min: string (nullable = true)
 |-- Bwd Pkt Len Mean: string (nullable = true)
 |-- Bwd Pkt Len Std: string (nullable = true)
 |-- Flow Byts/s: string (nullable = true)
 |-- Flow Pkts/s: string (nullable =

In [3]:
from pyspark.sql.functions import col
from pyspark.ml.feature import QuantileDiscretizer
from pyspark.sql.types import IntegerType

df = df.where(col("Flow Duration").isNotNull())
df = df.withColumn("Flow Duration",col("Flow Duration").cast(IntegerType()))
discretizer = QuantileDiscretizer(numBuckets=5, inputCol="Flow Duration", outputCol="flow_dur")

df = discretizer.fit(df).transform(df)
#df.show()

In [ ]:
import sys
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install pandas

In [4]:
from pyspark.sql.functions import split, explode, col, desc
from pyspark.ml import Pipeline 
from pyspark.ml.feature import VectorAssembler 
from pyspark.ml.feature import StringIndexer 
from pyspark.ml.classification import NaiveBayes 
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

indexers = [
StringIndexer(inputCol="Dst IP", outputCol = "dstip_index"),
StringIndexer(inputCol="Dst Port", outputCol = "dstport_index"),
StringIndexer(inputCol="FIN Flag Cnt", outputCol = "fin_index"),
StringIndexer(inputCol="SYN Flag Cnt", outputCol = "syn_index"),
StringIndexer(inputCol="URG Flag Cnt", outputCol = "urg_index"),
StringIndexer(inputCol="PSH Flag Cnt", outputCol = "cwe_index"),
StringIndexer(inputCol="ECE Flag Cnt", outputCol = "ece_index"),
#StringIndexer(inputCol="flow_dur", outputCol = "flowdur_index"),
StringIndexer(inputCol="Label", outputCol = "label")]

In [5]:
pipeline = Pipeline(stages=indexers)
df = df.select("Dst IP", 
               "Dst Port",
               "FIN Flag Cnt", 
               "SYN Flag Cnt", 
               "URG Flag Cnt",
               "PSH Flag Cnt",
               "ECE Flag Cnt",
#               "flow_dur",
               "Label"
              )
#df = df.sample(fraction=0.3)
indexed_rows_df = pipeline.fit(df).transform(df)
#indexed_rows_df.show(5, False)

22/02/17 21:02:22 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [9]:
vectorAssembler = VectorAssembler(inputCols = ["dstip_index","dstport_index","fin_index", "syn_index", "urg_index", "cwe_index","ece_index"],outputCol = "features")
vindexed_rows_df = vectorAssembler.transform(indexed_rows_df)
#vindexed_rows_df.show(5, False)

In [7]:
splits = vindexed_rows_df.randomSplit([0.6,0.4], 42) 
# optional value 42 is seed for sampling 
train_df = splits[0] 
test_df = splits[1]

In [8]:
nb = NaiveBayes(modelType="multinomial")
nbmodel = nb.fit(train_df)

22/02/17 21:12:22 WARN DAGScheduler: Broadcasting large task binary with size 3.1 MiB
22/02/17 21:13:50 WARN DAGScheduler: Broadcasting large task binary with size 3.1 MiB


In [10]:
predictions_df = nbmodel.transform(test_df)
#predictions_df.show(5, True)

In [11]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy") 
nbaccuracy = evaluator.evaluate(predictions_df) 
print("Test accuracy = " + str(nbaccuracy))
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")
nbf1 = evaluator.evaluate(predictions_df) 
print("F1 score = " + str(nbf1))

22/02/17 21:14:07 WARN DAGScheduler: Broadcasting large task binary with size 3.1 MiB


Test accuracy = 0.8377855007425652


22/02/17 21:15:36 WARN DAGScheduler: Broadcasting large task binary with size 3.1 MiB


F1 score = 0.8365096503902231
